## Video Tutorial

This EDA comes along with a video tutorial, check it out [here](https://www.youtube.com/watch?v=iiwEW-sg9KE&list=PL_49VD9KwQ_OJCqZOeOlSUQKcr1MyifOc&index=2).

In [1]:
# !pip install datasets
# !pip uninstall fsspec -y
# !pip install fsspec==2021.5.0

!pip install \
    /kaggle/input/huggingfaces/datasets/datasets* \
    /kaggle/input/huggingfaces/datasets/huggingface_hub* \
    /kaggle/input/huggingfaces/datasets/tqdm* \
    /kaggle/input/huggingfaces/datasets/xxhash*
!pip uninstall fsspec -y
!pip install /kaggle/input/huggingfaces/datasets/fsspec*

Processing /kaggle/input/huggingfaces/datasets/datasets-1.6.2-py3-none-any.whl
Processing /kaggle/input/huggingfaces/datasets/huggingface_hub-0.0.9-py3-none-any.whl
Processing /kaggle/input/huggingfaces/datasets/tqdm-4.49.0-py2.py3-none-any.whl
Processing /kaggle/input/huggingfaces/datasets/xxhash-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
Found existing installation: fsspec 0.8.7
Uninstalling fsspec-0.8.7:
  Successfully uninstalled fsspec-0.8.7
Processing /kaggle/input/huggingfaces/datasets/fsspec-2021.5.0-py3-none-any.whl


In [2]:
%env WANDB_DISABLED=true

env: WANDB_DISABLED=true


In [3]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch
from torch import nn

sns.set()
%matplotlib inline

In [4]:
data_dir = '/kaggle/input/commonlitreadabilityprize'
train_data_path = os.path.join(data_dir, 'train.csv')
test_data_path = os.path.join(data_dir, 'test.csv')

train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

print(len(train_df))
print(len(test_df))

2834
7


In [5]:
huggingface_dir = '/kaggle/input/huggingface-bert'
model_dir = os.path.join(huggingface_dir, 'bert-base-cased')

tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)

Some weights of the model checkpoint at /kaggle/input/huggingface-bert/bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [6]:
model.classifier = nn.Linear(768, 1)
model.num_labels = 1

In [7]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([0]).unsqueeze(0)  # Batch size 1
inputs['labels'] = labels
outputs = model(**inputs)
loss = outputs.loss
logits = outputs.logits

print(outputs)
print(loss)
print(logits)

SequenceClassifierOutput(loss=tensor(0.0921, grad_fn=<MseLossBackward>), logits=tensor([[-0.3035]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
tensor(0.0921, grad_fn=<MseLossBackward>)
tensor([[-0.3035]], grad_fn=<AddmmBackward>)


In [8]:
train_datasets = load_dataset('csv', data_files=[train_data_path])
test_datasets = load_dataset('csv', data_files=[test_data_path])

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-940d182d8e39bcc8/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-15ac625cb2155020/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [9]:
def tokenize_function(examples):
    return tokenizer(examples['excerpt'], padding='max_length', truncation=True, max_length=512)

f_train_datasets = train_datasets.map(tokenize_function, batched=True)
f_train_datasets = f_train_datasets.remove_columns(['id', 'url_legal', 'license', 'excerpt', 'standard_error'])
f_train_datasets = f_train_datasets.rename_column('target', 'labels')
f_train_datasets = f_train_datasets.shuffle(seed=42)

f_test_datasets = test_datasets.map(tokenize_function, batched=True)
f_test_datasets = f_test_datasets.remove_columns(['url_legal', 'license', 'excerpt'])

In [10]:
n_samples = len(f_train_datasets['train'])
n_train = int(0.9 * n_samples)

f_train_dataset = f_train_datasets['train'].select(range(n_train))
f_eval_dataset = f_train_datasets['train'].select(range(n_train, n_samples))

f_test_dataset = f_test_datasets['train']

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits, labels = logits.squeeze(), labels.squeeze()
    rmse = np.sqrt(np.mean((labels - logits) ** 2))
    return {'RMSE': rmse}

In [12]:
# os.environ['WANDB_API_KEY'] = '19baf7fe1571ebd98eff8449df8e8cbc3d30c634'

In [13]:
training_args = TrainingArguments(
    'training_args',
    num_train_epochs = 5,
    logging_steps = 200,
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    evaluation_strategy = 'steps'
)

trainer = Trainer(
    model = model,
    train_dataset = f_train_dataset,
    eval_dataset = f_eval_dataset,
    compute_metrics = compute_metrics,
    args = training_args
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
# for x in f_train_dataset:
#     a = len(x['input_ids'])
#     print(a)

# # [len(v) for v in f_train_dataset[0].values()]

In [15]:
trainer.train()

Step,Training Loss,Validation Loss,Rmse,Runtime,Samples Per Second
200,0.655800,0.414376,0.643721,5.277800,53.810000
400,0.489000,0.490163,0.700117,5.239600,54.203000
600,0.337800,0.426583,0.653133,5.254900,54.045000
800,0.252000,0.496258,0.704456,5.250100,54.094000
1000,0.182500,0.402305,0.634275,5.246700,54.130000
1200,0.132000,0.394221,0.627871,5.244600,54.151000
1400,0.089100,0.452221,0.672474,5.245200,54.145000


TrainOutput(global_step=1595, training_loss=0.2757031521453379, metrics={'train_runtime': 973.6802, 'train_samples_per_second': 1.638, 'total_flos': 4242326853888000.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 1778094080, 'init_mem_gpu_alloc_delta': 433773056, 'init_mem_cpu_peaked_delta': 88760320, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 259338240, 'train_mem_gpu_alloc_delta': 1361674752, 'train_mem_cpu_peaked_delta': 267329536, 'train_mem_gpu_peaked_delta': 6504080384})

In [16]:
trainer.evaluate()

{'eval_loss': 0.38351696729660034,
 'eval_RMSE': 0.6192874312400818,
 'eval_runtime': 5.2852,
 'eval_samples_per_second': 53.735,
 'epoch': 5.0,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_peaked_delta': 289528832}

In [17]:
# model.save_pretrained('model_v1')

In [18]:
pred_output = trainer.predict(f_test_dataset)
pred_targets = pred_output.predictions.squeeze()
pred_ids = f_test_dataset['id']

submission = pd.DataFrame({
    'id': pred_ids,
    'target': pred_targets
})

submission.to_csv('submission.csv', index=False)